In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Charger les données CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Vérifier les dimensions des ensembles de données
print("Dimensions des ensembles de données :")
print("Entraînement - Images :", x_train.shape, "Étiquettes :", y_train.shape)
print("Test - Images :", x_test.shape, "Étiquettes :", y_test.shape)

170498071/170498071 [==============================] - 2s 0us/step
Dimensions des ensembles de données :
Entraînement - Images : (50000, 32, 32, 3) Étiquettes : (50000, 1)
Test - Images : (10000, 32, 32, 3) Étiquettes : (10000, 1)


In [ ]:
# Convertir les valeurs des pixels en flottants et les mettre à l'échelle entre 0 et 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Vérifier les nouvelles valeurs des pixels
print("Valeurs des pixels après mise à l'échelle :")
print("Valeur minimale :", x_train.min())
print("Valeur maximale :", x_train.max())


Valeurs des pixels après mise à l'échelle :
Valeur minimale : 0.0
Valeur maximale : 1.0


In [ ]:
from sklearn.model_selection import train_test_split

# Diviser l'ensemble d'entraînement en un ensemble étiqueté et non étiqueté
x_train_labeled, x_train_unlabeled, y_train_labeled, y_train_unlabeled = train_test_split(x_train, y_train, test_size=0.8, random_state=42)

# Vérifier les dimensions des ensembles de données résultants
print("Dimensions des ensembles de données étiquetés :")
print("Images :", x_train_labeled.shape, "Étiquettes :", y_train_labeled.shape)
print("Dimensions des ensembles de données non étiquetés :")
print("Images :", x_train_unlabeled.shape, "Étiquettes :", y_train_unlabeled.shape)


Dimensions des ensembles de données étiquetés :
Images : (10000, 32, 32, 3) Étiquettes : (10000, 1)
Dimensions des ensembles de données non étiquetés :
Images : (40000, 32, 32, 3) Étiquettes : (40000, 1)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

# Définir l'architecture de l'auto-encodeur
input_img = Input(shape=(32, 32, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)  # Ajout d'une couche convolutive
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Créer le modèle de l'auto-encodeur
autoencoder = Model(input_img, decoded)

# Compiler le modèle
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
# Entraîner l'auto-encodeur
autoencoder.fit(x_train_unlabeled, x_train_unlabeled,
                epochs=8,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/8
313/313 [==============================] - 230s 728ms/step - loss: 0.5944 - val_loss: 0.5765
Epoch 2/8
313/313 [==============================] - 225s 720ms/step - loss: 0.5736 - val_loss: 0.5749
Epoch 3/8
313/313 [==============================] - 225s 721ms/step - loss: 0.5704 - val_loss: 0.5697
Epoch 4/8
313/313 [==============================] - 225s 719ms/step - loss: 0.5678 - val_loss: 0.5673
Epoch 5/8
313/313 [==============================] - 233s 746ms/step - loss: 0.5659 - val_loss: 0.5652
Epoch 6/8
313/313 [==============================] - 226s 723ms/step - loss: 0.5641 - val_loss: 0.5642
Epoch 7/8
313/313 [==============================] - 227s 725ms/step - loss: 0.5631 - val_loss: 0.5636
Epoch 8/8
313/313 [==============================] - 225s 721ms/step - loss: 0.5625 - val_loss: 0.5630


In [ ]:
# Obtenir les caractéristiques latentes des images non étiquetées
features_unlabeled = autoencoder.predict(x_train_unlabeled)

# Vérifier les dimensions des caractéristiques
print("Dimensions des caractéristiques latentes des images non étiquetées :", features_unlabeled.shape)

1250/1250 [==============================] - 21s 17ms/step
Dimensions des caractéristiques latentes des images non étiquetées : (40000, 32, 32, 3)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Créer le modèle de classification
classifier = Sequential([
    Flatten(input_shape=features_unlabeled.shape[1:]),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compiler le modèle
classifier.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

# Entraîner le modèle
classifier.fit(features_unlabeled, y_train_unlabeled, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10
250/250 [==============================] - 12s 44ms/step - loss: 2.1862 - accuracy: 0.1878 - val_loss: 1.9689 - val_accuracy: 0.3018
Epoch 2/10
250/250 [==============================] - 11s 44ms/step - loss: 2.0425 - accuracy: 0.2290 - val_loss: 1.9516 - val_accuracy: 0.3090
Epoch 3/10
250/250 [==============================] - 11s 43ms/step - loss: 2.0187 - accuracy: 0.2407 - val_loss: 1.9500 - val_accuracy: 0.3302
Epoch 4/10
250/250 [==============================] - 10s 38ms/step - loss: 2.0089 - accuracy: 0.2418 - val_loss: 1.9476 - val_accuracy: 0.3221
Epoch 5/10
250/250 [==============================] - 10s 41ms/step - loss: 1.9906 - accuracy: 0.2470 - val_loss: 1.9648 - val_accuracy: 0.3327
Epoch 6/10
250/250 [==============================] - 10s 40ms/step - loss: 1.9754 - accuracy: 0.2555 - val_loss: 1.9707 - val_accuracy: 0.2999
Epoch 7/10
250/250 [==============================] - 11s 43ms/step - loss: 1.9712 - accuracy: 0.2594 - val_loss: 1.9409 - val_accuracy:

In [ ]:
# Évaluer les performances du modèle sur l'ensemble de test
test_loss, test_accuracy = classifier.evaluate(x_test, y_test)

print("Perte sur l'ensemble de test :", test_loss)
print("Exactitude sur l'ensemble de test :", test_accuracy)

313/313 [==============================] - 2s 5ms/step - loss: 1.8919 - accuracy: 0.3411
Perte sur l'ensemble de test : 1.8919435739517212
Exactitude sur l'ensemble de test : 0.3411000072956085


In [ ]:
# Créer un nouveau modèle de classification pour les images originales
classifier_original = Sequential([
    Flatten(input_shape=(32, 32, 3)),  # Aplatir les images originales
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])

# Compiler le modèle
classifier_original.compile(optimizer='adam',
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])

# Entraîner le modèle sur les images originales
classifier_original.fit(x_train_labeled, y_train_labeled, epochs=10, batch_size=128, validation_split=0.2)

Epoch 1/10
63/63 [==============================] - 1s 14ms/step - loss: 2.2014 - accuracy: 0.2054 - val_loss: 2.0024 - val_accuracy: 0.2785
Epoch 2/10
63/63 [==============================] - 1s 11ms/step - loss: 1.9498 - accuracy: 0.3030 - val_loss: 1.8983 - val_accuracy: 0.3210
Epoch 3/10
63/63 [==============================] - 1s 11ms/step - loss: 1.8694 - accuracy: 0.3335 - val_loss: 1.9366 - val_accuracy: 0.3015
Epoch 4/10
63/63 [==============================] - 1s 11ms/step - loss: 1.8351 - accuracy: 0.3459 - val_loss: 1.8493 - val_accuracy: 0.3340
Epoch 5/10
63/63 [==============================] - 1s 11ms/step - loss: 1.7848 - accuracy: 0.3674 - val_loss: 1.8059 - val_accuracy: 0.3645
Epoch 6/10
63/63 [==============================] - 1s 11ms/step - loss: 1.7695 - accuracy: 0.3783 - val_loss: 1.7730 - val_accuracy: 0.3740
Epoch 7/10
63/63 [==============================] - 1s 11ms/step - loss: 1.7376 - accuracy: 0.3824 - val_loss: 1.7819 - val_accuracy: 0.3645
Epoch 8/10
63

In [ ]:
# Évaluer les performances du modèle sur l'ensemble de test
test_loss_original, test_accuracy_original = classifier_original.evaluate(x_test, y_test)

print("Perte sur l'ensemble de test avec les images originales :", test_loss_original)
print("Exactitude sur l'ensemble de test avec les images originales :", test_accuracy_original)

313/313 [==============================] - 1s 3ms/step - loss: 1.7476 - accuracy: 0.3817
Perte sur l'ensemble de test avec les images originales : 1.7475665807724
Exactitude sur l'ensemble de test avec les images originales : 0.3817000091075897
